# Final Project

## 파이썬 환경 설정 및 확인

In [ ]:
# C:/Users/TECH2_22/이도원/usingcloud_final/venv/Scripts/python.exe

In [ ]:
import os

os.getcwd()

In [ ]:
print('hello world')

In [ ]:
!python --version

## 데이터 로드 및 설정

### 최초 1회만 실행

In [1]:
import pandas as pd
import numpy as np
import json

with open('./data/train_original.json', encoding='utf-8') as train_f:
    train_data = json.loads(train_f.read())
    train_df = pd.DataFrame(train_data['documents'])

with open('./data/valid_original.json', encoding='utf-8') as valid_f:
    valid_data = json.loads(valid_f.read())
    valid_df = pd.DataFrame(valid_data['documents'])

In [2]:
def extract_body(article) -> str:
    art_sentence = []

    for contents in article:
        if len(contents) >= 2:
            for sub_con in contents:
                art_sentence.append(sub_con['sentence'])
                continue
        elif len(contents) == 0:
            pass
        else:
            art_sentence.append(contents[0]['sentence'])

    return art_sentence

def sentence_validation(art_sentence):
    del_sentence = []
    for sentence in art_sentence:
        if '@' in sentence or '/사진' in sentence:
            del_sentence.append(sentence)
        elif sentence[-1] != '.':
            del_sentence.append(sentence)
    for del_sen in del_sentence: 
        art_sentence.remove(del_sen)
    return ' '.join(art_sentence)

## eda

In [ ]:
train_df.head(3)

In [ ]:
train_df.info()

In [ ]:
train_df.shape

In [ ]:
train_df.head(1).T

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.columns

## 전처리

In [ ]:
def extract_body(article) -> str:
    art_sentence = []

    for contents in article:
        if len(contents) >= 2:
            for sub_con in contents:
                art_sentence.append(sub_con['sentence'])
                continue
        elif len(contents) == 0:
            pass
        else:
            art_sentence.append(contents[0]['sentence'])

    return art_sentence

def sentence_validation(art_sentence):
    del_sentence = []
    for sentence in art_sentence:
        if '@' in sentence or '/사진' in sentence:
            del_sentence.append(sentence)
        elif sentence[-1] != '.':
            del_sentence.append(sentence)
    for del_sen in del_sentence: 
        art_sentence.remove(del_sen)
    return ' '.join(art_sentence)

In [10]:
sentence_list = sentence_validation(extract_body(train_df['text'][24000]))
sentence_list

'은행 열매 수집망. 경기도내 지자체들이 매년 반복되는 가을철 은행나무 악취 민원 해결을 위해 온갖 방법을 동원하면서 시민들의 불편을 줄이고자 안간힘을 쓰고 있다. 6일 도내 지자체들에 따르면 친환경적인 방법으로 은행열매를 제거해 가을철 도로변 악취 문제를 해결하기 위한 방법을 고안하고 있다. 대표적으로 수원시는 산하 사업소를 통해 주요 대로변, 상가밀집지역 등 유동인구가 많은 곳을 대상으로 ‘은행 암나무 수종(樹種) 교체사업’을 진행한다. 시는 지난해 은행나무를 전수조사하고 수종 교체를 위한 연차별 기본계획을 수립한 바 있다. 지난 4월 사업을 시작해 은행 암나무 600여 그루를 은행 수나무·느티나무 등 다른 나무로 교체했다. 2022년까지 예산 36억 원을 투입할 예정이다. 아직 교체하지 못 했거나 보존 가치가 있는 은행나무는 열매 털기 작업을 실시한다. 은행열매 진동 수확기를 활용해 은행열매를 털어 악취를 없앤다. 은행나무 열매와 낙엽을 한꺼번에 처리할 수 있는 ‘은행열매 수집망’도 설치했다. 은행열매 수집망은 나무에 해를 가하지 않고 열매와 낙엽을 제거하는 친환경 방법이다. 현재 도로 주행 차량에 방해를 주지 않는 팔달로와 창룡대로에 9개 수집망을 설치했다. 안산시 단원구는 지난달 21일부터 은행나무 열매로 인한 지속적인 악취로 고통받는 시민 피해를 막기 위해 은행열매를 조기 채취하고 있다. 원선1로 등 3개 노선은 수거를 완료했으며, 휴일에도 3개 조 15명을 투입해 평일 작업이 어려운 산업단지 등의 열매를 제거하고 있다. 구는 수거된 은행열매에 대해 중금속검사와 잔류농약검사를 벌여 ‘안전하다’고 판정된 수확물은 지역 복지관 및 경로당에 기부할 예정이다. 또한 은행열매 수집망 설치도 검토하는 등 시민 불편을 줄일 수 있도록 다양한 방안을 모색 중이다. 부천시와 고양시 덕양구 역시 은행나무 암나무를 열매를 맺지 않는 수나무로 바꾼다. 부천시는 먼저 126그루를 수나무로 단계적 교체한다. 주민 피해가 심한 도심부터 작업을 추진해 순차적으로 1천200그루를

In [3]:
new_df = train_df.copy()
new_df.text = train_df.text.apply(lambda x : sentence_validation(extract_body(x)))
new_df.head(3)

,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive
0,290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,11,"{'readable': 4, 'accurate': 3, 'informative': ...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...
1,290741792,종합,online,지역지,광양신문,small,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성",여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 4, 11]",[여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천...
2,290741793,스포츠,online,지역지,광양신문,medium,1066,2018-01-05 18:52:15,“새해 정기 받고 올해는 반드시 일내자!”,전남드래곤즈(사장 신승재)는 지난 4일 구봉산 해맞이 행사를 통해 새해 각오를 다졌...,13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 5, 7]",[전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구...


In [6]:
from tokenization_kobert import KoBertTokenizer
import sentencepiece as spm

# sentencepiece 라이브러리 설치 필요
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [31]:
print(len(tokenizer.tokenize(new_df.text[0])))

470


In [16]:
print(tokenizer.convert_ids_to_tokens(tokenizer.encode(new_df.text[0])))

['[CLS]', '▁전', '라', '남', '도', '가', '▁', '쌀', '▁', '과', '잉', '문제', '를', '▁근본', '적으로', '▁해결', '하기', '▁위해', '▁올해부터', '▁시행', '하는', '▁', '쌀', '▁생산', '조정', '제를', '▁적극', '▁추진', '키로', '▁했다', '.', '▁', '쌀', '▁생산', '조정', '제', '는', '▁', '벼', '를', '▁심', '었던', '▁논', '에', '▁', '벼', '▁대신', '▁사', '료', '작', '물', '이나', '▁', '콩', '▁등', '▁다른', '▁작', '물을', '▁심', '으면', '▁', '벼', '와의', '▁일정', '▁소득', '차', '를', '▁보', '전', '해주는', '▁제도', '다', '.', '▁올해', '▁전남', '의', '▁논', '▁다른', '▁작', '물', '▁재', '배', '▁계획', '면', '적', '은', '▁전국', '▁5', '만', 'ha', '의', '▁약', '▁2', '1%', '인', '▁1', '만', '▁6', '98', 'ha', '로', ',', '▁세부', '시', '행', '지', '침', '을', '▁확정', ',', '▁시', '군', '에', '▁통보', '했다', '.', '▁지원', '사업', '▁대상', '은', '▁2017', '년', '산', '▁', '쌀', '▁변동', '직', '불', '금을', '▁받은', '▁농', '지', '에', '▁10', 'a', '(', '300', '평', ')', '▁이상', '▁', '벼', '▁이', '외', '▁다른', '▁작', '물을', '▁재', '배', '한', '▁농업', '인', '이다', '.', '▁지원', '▁대상', '▁작', '물', '은', '▁1', '년생', '을', '▁포함한', '▁다', '년생', '의', '▁모든', '▁작', '물', '이', '▁해당', '되', '나

In [6]:
# 길이가 가장 긴 텍스트 탐색
# index : 66068
# 길이 : 13910
new_df.text.str.len().sort_values(ascending=False)

66068     13910
36893      1963
182051     1959
187153     1955
105903     1954
          ...  
65237         0
157929        0
157927        0
66            0
156078        0
Name: text, Length: 243983, dtype: int64

In [7]:
new_df.to_csv('./data/refined_data.csv', index=False)

In [22]:
len(tokenizer.tokenize(new_df['text'][66068]))

7422

In [27]:
SEQ_LEN = 7500

len(tokenizer.encode(new_df['text'][66068], max_length=SEQ_LEN, padding='max_length'))

7500

In [29]:
result = tokenizer.encode_plus(new_df['text'][0], max_length=SEQ_LEN, padding="max_length")
result

{'input_ids': [2, 4012, 6003, 5666, 5859, 5330, 517, 6756, 517, 5468, 7144, 6236, 6116, 1226, 7203, 4999, 7789, 3567, 3451, 3004, 7794, 517, 6756, 2714, 7259, 7236, 3997, 4555, 7574, 5019, 54, 517, 6756, 2714, 7259, 7234, 5760, 517, 6353, 6116, 3060, 6889, 1498, 6896, 517, 6353, 1655, 2573, 6093, 7170, 6241, 7098, 517, 7546, 1815, 1567, 3930, 6242, 3060, 7083, 517, 6353, 6984, 3820, 2828, 7389, 6116, 2355, 7207, 7857, 4137, 5782, 54, 3450, 4018, 7095, 1498, 1567, 3930, 6241, 3969, 6312, 990, 6198, 7202, 7086, 4014, 611, 6150, 402, 7095, 3211, 553, 94, 7119, 529, 6150, 617, 247, 402, 6079, 46, 2809, 6705, 7881, 7318, 7491, 7088, 5134, 46, 2959, 5512, 6896, 4746, 7869, 54, 4331, 6503, 1651, 7086, 578, 5712, 6516, 517, 6756, 2341, 7342, 6424, 5557, 2232, 1507, 7318, 6896, 533, 367, 18, 145, 7724, 40, 3704, 517, 6353, 3647, 6995, 1567, 3930, 6242, 3969, 6312, 7828, 1508, 7119, 7100, 54, 4331, 1651, 3930, 6241, 7086, 529, 5718, 7088, 4868, 1562, 5718, 7095, 2048, 3930, 6241, 7096, 5000, 588

In [5]:
from tqdm import tqdm
from tokenization_kobert import KoBertTokenizer

import re
import numpy as np
import sentencepiece as spm

# sentencepiece 라이브러리 설치 필요
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

SEQ_LEN = 7500

token_ids = []
token_masks = []
token_seqments = []

train_labels = []

for idx in tqdm(range(len(new_df.text))):
    train_sentence = new_df.text.iloc[idx]

    cleaned_sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", train_sentence) # [ whitespaces, 숫자, 영문 알파벳, 한글(+자모음) ]이 아닌 것을 공백으로 치환 (특수문자 제거)

    encoded_dict = tokenizer.encode_plus(
        text = cleaned_sentence,
        padding = 'max_length',
        truncation = True,
        max_length = SEQ_LEN,
    )

    token_ids.append(encoded_dict['input_ids'])
    token_masks.append(encoded_dict['token_type_ids'])
    token_seqments.append(encoded_dict['attention_mask'])

train_inputs = (np.array(token_ids), np.array(token_masks), np.array(token_seqments))
train_inputs

c:\Users\TECH2_22\이도원\usingcloud_final\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.
  1%|▏         | 3176/243983 [00:07<08:55, 450.07it/s]


KeyboardInterrupt: 

In [52]:
np.array(token_ids)

MemoryError: Unable to allocate 6.82 GiB for an array with shape (243983, 7500) and data type int32

In [55]:
import sys

local_vars = list(locals().items())

for var, obj in local_vars:
    obj_size = sys.getsizeof(obj) # Return the size of object in bytes -> 1064 * 1064 bytes == 1 MB
    if (obj_size > 1064 * 1064) and (not var.startswith('_')): 
        print(var, int(obj_size / (1064 * 1064)), 'MB')

sys.getwindowsversion(major=10, minor=0, build=19044, platform=2, service_pack='')

In [44]:
new_df.text.iloc[0]

'전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.'

## 여기가 끝